# Computational and Artifical Intelligenge Assignment 1

## Problem Solving by Searching using IDA* and Best First Search
---
Weightage: 10%

Date of Posting - 04/July/2019

Deadline - 22/July/2019 10:00 AM

## Submitters 

### Group ID/Number: S2-18_DSECL_ACI_BAN_2
---

<table style="float:left">
    <tbody>
        <tr>
            <td>1</td>
            <td>Nageswar Rao</td>
            <td>2018ab04093 </td>
        </tr>
         <tr>
            <td>2</td>
            <td>Saranya</td>
            <td>2018ab04102</td>
        </tr>
         <tr>
            <td>3</td>
            <td>Prabha Sasikanth</td>
            <td>2018AB04094</td>
        </tr>
    </tbody>
</table>

## Table of Contents  <span id="0" ></span>
---
1. [**Problem statement**](#1)
2. [Environment representation](#2)
3. [Frindge datastructure](#3)
4. [IDA* algorithm](#4)
5. [Best First Search Algorithm](#5)
6. [Search Execution](#6)
7. [Results](#7)

Things to follow

1. Use appropriate data structures for the fringes and explain the reasoning behind the usage. You need not create data structures, instead use available libraries directly.

2. Avoid any hard-coding unless absolutely necessary.

3. Provide proper documentation

<span id="1"></span>
## The problem statement
---
[[top]](#0)

The agent stranded somewhere on an 11x8 grid. The green dot in the below figure represents your present location on the grid. The agent is low on food and water and you are running out of energy to survive. Going from one cell to the next consumes one unit of your precious energy. Needless to say, you have an urgent need to exit this grid in as few steps as possible. There is only one cell through which you can exit the grid and that cell is represented by the blue arrow. The grid also has some barriers which agent cannot cross but can go around. Agent can only move right, left, up and down to a cell which is within the grid and which is not a barrier.

The environment is static, fully observable and deterministic

<img src="grid.png">

In order to use one of the informed search strategies, 
    
1. which heuristic function h(n) would you use? Is your heuristic both admissible and consistent? Please explain.
    
2. How will you go about defining the total cost function f(n)

3. Implement Iterative deepening A*(IDA*) search ++ in python to find the path to the exit cell avoiding the barriers. Your output should contain the path taken by the agent (e.g. [(3,8),(4,8),(4,7)………] and also the total cost incurred in taking that path, total number of nodes expanded to get this state, maximum number of nodes kept at the memory at any point in time

4. Implement Greedy Best First Search in python to find the path to the exit cell avoiding the barriers. Your output should contain the path taken by the agent (e.g. [(3,8),(4,8),(4,7)………] and also the total cost incurred in taking that path, total number of nodes expanded to get this state,
maximum number of nodes kept at the memory at any point in time

5. Compare the following for both IDA* and Best First Search
    - Path taken to exit the grid
    - Cost of this path. Is the path optimal for both?
    - Total Number of nodes expanded to get this state [time complexity]
    - Maximum number of nodes kept at the memory at any point in time [space complexity]


Coding begins now!!!

<span id="2"></span>
## Environment representation
---
[[top]](#0)

### Maze
Input maze is defined as a two dimensional matrix which is represented by (0,0) to (7,10). The cell value '1' represent the agent can move freely, where as '0' is the obstacle/block

### Node
The class represents the class to hold the variables required to represent a cell in the maze

In [35]:
import queue
import math

#Environment representation goes here
inputMaze = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1],
             [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
             [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
             [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
             [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
             [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        # Reference to the parent node
        self.parent = parent
        # Position of the current node
        self.position = position

        # Actual cost to reach the current node from the initial node
        self.g = 0
        # Heuristic, an estimate to reach destination from current node
        self.h = 0
        # Summation of cost and heuristic
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def __cmp__(self, other):
        return cmp(self.f, other.f)

<span id="3"></span>
## Fringe data structure
---
[[top]](#0)

Define the fringe (data structure) and its supported methods in the following code block

In [36]:
#Fringe representation is a priority queue and its associated methods
class PriorityQueue(object): 
    def __init__(self): 
        self.queue = [] 
  
    def __str__(self): 
        return ' '.join([str(i) for i in self.queue]) 
  
    # for checking if the queue is empty 
    def empty(self): 
        return self.queue == [] 
  
    # for inserting an element in the queue 
    def append(self, data): 
        self.queue.append(data) 
  
    # for inserting an element in the queue 
    def length(self): 
        return len(self.queue) 

    # for reading an element in the queue 
    def get(self, index): 
        return self.queue[index]

    # for popping an element based on Priority 
    def pop(self): 
        try: 
            max = 0
            for i in range(len(self.queue)): 
                if self.queue[i].f < self.queue[max].f: 
                    max = i 
            item = self.queue[max] 
            del self.queue[max] 
            return item 
        except IndexError: 
            print() 
            exit()
            
def testPriorityQueue():
    q = PriorityQueue()
    n1 = Node(None, (0,0))
    n1.f=10

    n2 = Node(None, (0,0))
    n2.f=5

    n3 = Node(None, (0,0))
    n3.f = 20

    q.append(n1)
    q.append(n2)
    q.append(n3)

    while not q.empty():
        print(q.get().f)

<span id="4"></span>
## IDA* Algorithm
---
[[top]](#0)

Define your IDA* in the following code block

In [37]:
#Algorithm IDA*
import sys

# Class that defines data structure and supporting methods to run IDA* algorithm,
class IDAStarSearch:
    def __init__(self, maze, start, end):
        
        # An array of nodes to represent the traversal path
        self.solution = []
        
        # Instance variable of the maze
        self.maze = maze
        
        # Instance variable of the start node
        self.start_node = Node(None, start)
        
        # Instance variable of the end node
        self.end_node = Node(None, end)
        
        # Priority queue represents the open list
        self.open_list = PriorityQueue()
        
        # Queue represents the closed list
        self.closed_list = queue.Queue()

        # Set the heuristcs of start node
        self.start_node.f = self.start_node.h = self.getheuristic(self.start_node)

        # Add the current start node to the open list
        self.open_list.append(self.start_node)
        
    
    # Method to calculate heuristic value of the given node (manhattan distance to the end node)
    # It can be euclidean distance as well.
    #
    # The function is to underestimate distance to end node in the maze based on admisssible property of the A* algorithm
    def getheuristic(self, node):
        # Euclidean distance
        # ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
        # Return manhattan distance
        return  abs(node.position[0] - self.end_node.position[0]) + abs(node.position[1] - self.end_node.position[1])
    
    # Method to check if the given position is out side of the maze
    def isoutside(self, maze, position):
        return position[0] > (len(maze) - 1) or position[0] < 0 or position[1] > (len(maze[len(maze)-1]) -1) or position[1] < 0
    
    # Method for IDA* search
    def solve(self):
        
        # Ensure start node is inside the maze
        if self.isoutside(self.maze, self.start_node.position):
            return 'INVALID Start position'

        # Ensure end node is insize the mase
        if self.isoutside(self.maze, self.end_node.position):
            return 'INVALID End position'
        
        # start the bound with the heuristic of start node
        bound = self.getheuristic(self.start_node)

        # Search begins
        while True:
            
            # Pick the top node in open list and start search
            current_node = self.open_list.pop()
            self.closed_list.put(current_node)
            t = self.search(current_node, 0, bound)
            
            # If found return the solution path
            if t == 'FOUND':
                path = []
                current = self.solution[0]
                while current is not None:
                    path.append(current.position)
                    current = current.parent
                return path[::-1] # Return reversed path
            else:
                path = []
                return path[::-1] # Return reversed path

    # Recursive method for search
    def search(self, current_node, g, bound):

        # Bound is calculated based on heuristic
        f = g + self.getheuristic(current_node)
        if f > bound:
            return f

        # Match if heuristic of the end node
        if self.getheuristic(current_node) == 0:
            return 'FOUND'

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            
            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure boundaries are not crossed
            if self.isoutside(self.maze, node_position):
                continue

            # Make sure walkable terrain
            if self.maze[node_position[0]][node_position[1]] != 1:
                continue

            # Create new node
            neighbour = Node(current_node, node_position)
            
            # Append
            children.append(neighbour)

        # Loop through children and check if children are in closed_list
        for child in children:
            # Child is on the closed list
            for closed_child in list(self.closed_list.queue):
                if child == closed_child:
                    continue
            
            # Update heuristics of children
            child.h = self.getheuristic(child)
            child.f = child.h

            # Child is already in the open list
            for i in range(self.open_list.length()):
                open_node = self.open_list.get(i)
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            self.open_list.append(child)
        
        # Resetting the next threshold
        minimum = sys.maxsize
        next_node = self.open_list.pop()
        self.closed_list.put(next_node)
        t = self.search(next_node, g + 1, bound)
        if t == 'FOUND':
            self.solution.append(next_node)
            return 'FOUND'
        
        # set next threshold
        if t < minimum:
            minimum = t
        return minimum

<span id="5"></span>
## Best First Search Algorithm
---
[[top]](#0)

Define your Greedy Best First Search algorithm in the following code block

In [38]:
#Algorithm Best First Search

class BestFirstSearch:
    def __init__(self, maze, start, end):
        
        # An array of nodes to represent the traversal path
        self.solution = []
        
        # Instance variable of the maze
        self.maze = maze
        
        # Instance variable of the start node
        self.start_node = Node(None, start)
        
        # Instance variable of the end node
        self.end_node = Node(None, end)
        
        # Priority queue represents the open list
        self.open_list = PriorityQueue()
        
        # Queue represents the closed list
        self.closed_list = queue.Queue()

        # Set the heuristcs of start node
        self.start_node.f = self.start_node.h = self.getheuristic(self.start_node, self.end_node)

        # Add the current start node to the open list
        self.open_list.append(self.start_node)

    # Method to check if the given position is outside maze
    def isoutside(self, maze, position):
        return position[0] > (len(maze) - 1) or position[0] < 0 or position[1] > (len(maze[len(maze)-1]) -1) or position[1] < 0

    def getheuristic(self, node, end):
        # Euclidean distance
        # ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
        # Manhattan distance
        return (abs(node.position[0] - end.position[0]) + abs(node.position[1] - end.position[1]))

    def search(self):
        """Returns a list of tuples as a path from the given start to the given end in the given maze"""

        if self.isoutside(self.maze, self.start_node.position):
            return 'INVALID Start node'
    
        if self.isoutside(self.maze, self.end_node.position):
            return 'INVALID Start node'

        # Initialize both open and closed list
        self.open_list = PriorityQueue()
        self.closed_list = queue.Queue()

        # Add the start node
        self.open_list.append(self.start_node)
    
        # Loop until you find the end
        while self.open_list.length() > 0:

            # Get the current node
            current_node = self.open_list.pop()
            self.closed_list.put(current_node)

            # Found the goal
            if current_node == self.end_node:
                path = []
                current = current_node
                while current is not None:
                    path.append(current.position)
                    current = current.parent
                return path[::-1] # Return reversed path

            # Generate children
            children = []
            for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

                # Get node position
                node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

                # Make sure within range
                if self.isoutside(self.maze, node_position):
                    continue

                # Make sure walkable terrain
                if self.maze[node_position[0]][node_position[1]] != 1:
                    continue

                # Create new node
                new_node = Node(current_node, node_position)

                # Append
                children.append(new_node)

            # Loop through children
            for child in children:

                # Child is on the closed list
                for closed_child in list(self.closed_list.queue):
                    if child == closed_child:
                        continue

                # set the heuristic to sort in priority queue
                child.h = self.getheuristic(child, self.end_node)
                child.f = child.h

                # Child is already in the open list
                for i in range(self.open_list.length()):
                    open_node = self.open_list.get(i)
                    if child == open_node and child.g > open_node.g:
                        continue

                # Add the child to the open list
                self.open_list.append(child)

Feel free to add code blocks for any other methods needed starting here.

In [39]:
#Code Block 1

<span id="6"></span>
## Search Execution
---
[[top]](#0)

Call your main function/algorithm for IDA* in the next code block with appropriate input representation

In [40]:
#Computation call for IDA*
if __name__ == '__main__':
    print('Starting IDA* Algorithm')
    start = (0, 2)
    end = (5, 10)
    
    idastar = IDAStarSearch(inputMaze, start, end)
    idapath = idastar.solve()

Starting IDA* Algorithm


Call your main function/algorithm for Best First Search in the next code block with appropriate input representation

In [41]:
#Computation call for Best First Search
#Computation call for Best First Search
if __name__ == '__main__':
    print('Starting Best First Search')
    start = (0, 2)
    end = (5, 10)
    bfs = BestFirstSearch(inputMaze, start, end)
    bfspath = bfs.search()

Starting Best First Search


<span id="7"></span>
## Results
---
[[top]](#0)

The agent should provide expected output for questions mentioned below in the subsequent blocks

(1) Path taken to reach destination from source for IDA*

In [42]:
# Execute statement to retrieve the path taken here using IDA*
print("Traversal path by IDA*: ", idapath)

Traversal path by IDA*:  [(0, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 7), (5, 8), (5, 9), (5, 10)]


(1) Path taken to reach destination from source for Best First Search

In [43]:
# Execute statement to retrieve the path taken here using Best First Search
print("Traversal path by Best First Search: ", bfspath)

Traversal path by Best First Search:  [(0, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 7), (5, 8), (5, 9), (5, 10)]


(2) Cost of the path for IDA* here

In [44]:
# Execute statement to retrieve the cost of the path here using IDA*
print("Cost of the path using IDA*: ", len(idapath))

Cost of the path using IDA*:  9


(2) Cost of the path for Best First Search here

In [45]:
# Execute statement to retrieve the cost of the path here using Best First Search
print("Cost of the path using Best First Search: ", len(bfspath))

Cost of the path using Best First Search:  9


(3) Total Number of nodes expanded to get this state using IDA*

In [46]:
# Execute statement to retrieve the total number of nodes expanded to get this state using IDA* here
print("Total number of nodes expanded by IDA: ", idastar.open_list.length())

Total number of nodes expanded by IDA:  43


(3) Total Number of nodes expanded to get this state using Best First Search

In [47]:
# Execute statement to retrieve the total number of nodes expanded to get this state using Best First Search here
print("Total number of nodes expanded by Best First Search: ", bfs.open_list.length())

Total number of nodes expanded by Best First Search:  43


(4) Maximum number of nodes kept at the memory at any point in time using IDA*

In [48]:
# Execute Output for the maximum number of nodes kept at the memory at any point in time using IDA* here
print("Total number of nodes kept at the memory by IDA*: ", (idastar.open_list.length() + len(idastar.closed_list.queue)))

Total number of nodes kept at the memory by IDA*:  52


(4) Maximum number of nodes kept at the memory at any point in time using Best First Search

In [49]:
# Execute Output for the maximum number of nodes kept at the memory at any point in time using Best First Search here
print("Total number of nodes kept at the memory by BFS: ", (bfs.open_list.length() + len(bfs.closed_list.queue)))

Total number of nodes kept at the memory by BFS:  52


All the best!! Happy Coding!!
Let human intelligence prevail